# Random Forest 

In [1]:
import os 
import sys
import math

import pickle
import optuna
import logging
import numpy as np
import pandas as pd
import seaborn as sn
# from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve

%matplotlib inline

data_folder = '../data/'

### A) Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). 
- Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. 
- Standardize the values of each feature in the data to have mean 0 and variance 1.

In [2]:
columns = ['LU4', 'LC4', 'LU3', 'LC3', 'LU2', 'LC2', 'LU1', 'LC1', 'nbr1_LU3', 'nbr1_LC3', 'nbr1_LU2', 'nbr1_LC2', 'nbr1_LU1',
           'nbr1_LC1', 'nbr2_LU3', 'nbr2_LC3', 'nbr2_LU2', 'nbr2_LC2', 'nbr2_LU1', 'nbr2_LC1', 'nbr3_LU3', 'nbr3_LC3',
           'nbr3_LU2', 'nbr3_LC2', 'nbr3_LU1', 'nbr3_LC1', 'nbr4_LU3', 'nbr4_LC3', 'nbr4_LU2', 'nbr4_LC2', 'nbr4_LU1',
           'nbr4_LC1', 'nbr5_LU3', 'nbr5_LC3', 'nbr5_LU2', 'nbr5_LC2', 'nbr5_LU1', 'nbr5_LC1', 'nbr6_LU3', 'nbr6_LC3',
           'nbr6_LU2', 'nbr6_LC2', 'nbr6_LU1', 'nbr6_LC1', 'nbr7_LU3', 'nbr7_LC3', 'nbr7_LU2', 'nbr7_LC2', 'nbr7_LU1',
           'nbr7_LC1', 'nbr8_LU3', 'nbr8_LC3', 'nbr8_LU2', 'nbr8_LC2', 'nbr8_LU1', 'nbr8_LC1']

original_data = pd.read_csv(os.path.join(data_folder, 'trainset_with_neighbour.csv'), index_col=0)
original_data = original_data[columns]
original_data.head()

,LU4,LC4,LU3,LC3,LU2,LC2,LU1,LC1,nbr1_LU3,nbr1_LC3,...,nbr7_LU2,nbr7_LC2,nbr7_LU1,nbr7_LC1,nbr8_LU3,nbr8_LC3,nbr8_LU2,nbr8_LC2,nbr8_LU1,nbr8_LC1
RELI,,,,,,,,,,,,,,,,,,,,,
48561099,LU301,LC41,LU301,LC41,LU301,LC41,LU301,LC41,LU301,LC41,...,LU221,LC21,LU221,LC21,LU301,LC41,LU301,LC41,LU301,LC41
48611112,LU402,LC61,LU402,LC61,LU402,LC61,LU402,LC61,LU402,LC61,...,LU221,LC21,LU221,LC21,LU221,LC21,LU221,LC21,LU221,LC21
48621113,LU103,LC47,LU103,LC47,LU421,LC31,LU421,LC31,LU106,LC12,...,LU222,LC21,LU222,LC21,LU402,LC61,LU402,LC61,LU402,LC61
48621114,LU106,LC12,LU106,LC12,LU106,LC12,LU106,LC12,LU142,LC15,...,LU222,LC21,LU222,LC21,LU402,LC61,LU402,LC61,LU402,LC61
48621115,LU142,LC11,LU142,LC15,LU142,LC15,LU142,LC15,LU402,LC61,...,LU222,LC21,LU222,LC21,LU402,LC61,LU402,LC61,LU402,LC61


In [3]:
print('The length of the data with all rows is : {}'.format(len(original_data)))
original_data.dropna(inplace=True)
print('The length of the data without the rows with nan value is: {}'.format(len(original_data)))

The length of the data with all rows is : 348474
The length of the data without the rows with nan value is: 348474


In [4]:
data_features = original_data.copy()
data_features['changed'] = [0 if row['LU4'] == row['LU3'] and row['LC4'] == row['LC3'] else 1 for ind, row in data_features[['LU4', 'LC4', 'LU3', 'LC3']].iterrows()]
data_features.head()

,LU4,LC4,LU3,LC3,LU2,LC2,LU1,LC1,nbr1_LU3,nbr1_LC3,...,nbr7_LC2,nbr7_LU1,nbr7_LC1,nbr8_LU3,nbr8_LC3,nbr8_LU2,nbr8_LC2,nbr8_LU1,nbr8_LC1,changed
RELI,,,,,,,,,,,,,,,,,,,,,
48561099,LU301,LC41,LU301,LC41,LU301,LC41,LU301,LC41,LU301,LC41,...,LC21,LU221,LC21,LU301,LC41,LU301,LC41,LU301,LC41,0
48611112,LU402,LC61,LU402,LC61,LU402,LC61,LU402,LC61,LU402,LC61,...,LC21,LU221,LC21,LU221,LC21,LU221,LC21,LU221,LC21,0
48621113,LU103,LC47,LU103,LC47,LU421,LC31,LU421,LC31,LU106,LC12,...,LC21,LU222,LC21,LU402,LC61,LU402,LC61,LU402,LC61,0
48621114,LU106,LC12,LU106,LC12,LU106,LC12,LU106,LC12,LU142,LC15,...,LC21,LU222,LC21,LU402,LC61,LU402,LC61,LU402,LC61,0
48621115,LU142,LC11,LU142,LC15,LU142,LC15,LU142,LC15,LU402,LC61,...,LC21,LU222,LC21,LU402,LC61,LU402,LC61,LU402,LC61,1


In [5]:
print('Total number of tiles that changed label in either Land Cover or Land Usage: %d' % sum(data_features.changed))

Total number of tiles that changed label in either Land Cover or Land Usage: 58737


In [6]:
def split_set(data_to_split, ratio=0.8):
    mask = np.random.rand(len(data_to_split)) < ratio
    return [data_to_split[mask].reset_index(drop=True), data_to_split[~mask].reset_index(drop=True)]

In [7]:
[train, test] = split_set(data_features)

In [8]:
train_categorical = pd.get_dummies(train)
train_categorical.columns

Index(['changed', 'LU4_LU101', 'LU4_LU102', 'LU4_LU103', 'LU4_LU104',
       'LU4_LU105', 'LU4_LU106', 'LU4_LU107', 'LU4_LU108', 'LU4_LU121',
       ...
       'nbr8_LC1_LC45', 'nbr8_LC1_LC46', 'nbr8_LC1_LC47', 'nbr8_LC1_LC51',
       'nbr8_LC1_LC52', 'nbr8_LC1_LC53', 'nbr8_LC1_LC61', 'nbr8_LC1_LC62',
       'nbr8_LC1_LC63', 'nbr8_LC1_LC64'],
      dtype='object', length=2045)

In [9]:
# Make sure we use only the features available in the training set
test_categorical = pd.get_dummies(test)[train_categorical.columns]

In [10]:
train_label=train_categorical.changed
train_features = train_categorical.drop('changed', axis=1)
print('Length of the train dataset : {}'.format(len(train)))

test_label=test_categorical.changed
test_features = test_categorical.drop('changed', axis=1)
print('Length of the test dataset : {}'.format(len(test)))

Length of the train dataset : 278767
Length of the test dataset : 69707


### B) Train a random forest classifier on your training set. 

In [11]:
def compute_confusion_matrix(true_label, prediction_proba, decision_threshold=0.5, definition='normal'): 
    
    assert definition == 'normal' or definition == 'special'
    predict_label = (prediction_proba[:,1]>decision_threshold).astype(int)   
    
    if definition == 'normal':
        # normal definition of confusion matrix 
        TP = np.sum(np.logical_and(predict_label==1, true_label==1))
        TN = np.sum(np.logical_and(predict_label==0, true_label==0))
        FP = np.sum(np.logical_and(predict_label==1, true_label==0))
        FN = np.sum(np.logical_and(predict_label==0, true_label==1))
    
    elif definition == 'special':
        # special definition of the confusion matrix in this case to optimize the recall and precision of unchanged
        TP = np.sum(np.logical_and(predict_label==0, true_label==0))
        TN = np.sum(np.logical_and(predict_label==1, true_label==1))
        FP = np.sum(np.logical_and(predict_label==0, true_label==1))
        FN = np.sum(np.logical_and(predict_label==1, true_label==0))
    
    confusion_matrix = np.asarray([[TP, FP],
                                    [FN, TN]])
    return confusion_matrix


def plot_confusion_matrix(confusion_matrix, thred, axs):
    [[TP, FP],[FN, TN]] = confusion_matrix
    label = np.asarray([['TP {}'.format(TP), 'FP {}'.format(FP)],
                        ['FN {}'.format(FN), 'TN {}'.format(TN)]])
    
    df_cm = pd.DataFrame(confusion_matrix, index=['Yes', 'No'], columns=['Positive', 'Negative']) 
    
    sn.heatmap(df_cm, cmap='YlOrRd', annot=label, annot_kws={"size": 16}, cbar=False, fmt='',ax=axs)
    axs.set_xlabel('Actual')
    axs.set_ylabel('Predicted')
    axs.set_title('Confusion matrix for a {} threshold'.format(thred))
    


def compute_all_score(confusion_matrix, t=0.5):
    [[TP, FP],[FN, TN]] = confusion_matrix.astype(float)
    
    accuracy =  (TP+TN)/np.sum(confusion_matrix)
    
    precision_positive = TP/(TP+FP) if (TP+FP) !=0 else np.nan
    precision_negative = TN/(TN+FN) if (TN+FN) !=0 else np.nan
    
    recall_positive = TP/(TP+FN) if (TP+FN) !=0 else np.nan
    recall_negative = TN/(TN+FP) if (TN+FP) !=0 else np.nan

    F1_score_positive = 2 *(precision_positive*recall_positive)/(precision_positive+recall_positive) if (precision_positive+recall_positive) !=0 else np.nan
    F1_score_negative = 2 *(precision_negative*recall_negative)/(precision_negative+recall_negative) if (precision_negative+recall_negative) !=0 else np.nan

    return [t, accuracy, precision_positive, recall_positive, F1_score_positive, precision_negative, recall_negative, F1_score_negative]

Optimize the Random Forest Classifier with Optuna 

In [14]:
def objective(trial):
    '''
    Execute optuna and set hyperparameters
    '''
    criterion = trial.suggest_categorical('criterion', ["gini", "entropy", "log_loss"])
    bootstrap = trial.suggest_categorical('bootstrap',['True','False'])
    max_depth = trial.suggest_int('max_depth', 1, 1000)
    max_features = trial.suggest_categorical('max_features', [None, 'sqrt','log2'])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    n_estimators =  trial.suggest_int('n_estimators', 100, 1000, 100)
    
    regr = RandomForestClassifier(bootstrap = bootstrap, criterion = criterion,
                                  max_depth = max_depth, max_features = max_features,
                                  n_estimators = n_estimators, min_samples_split = min_samples_split,
                                  min_samples_leaf = min_samples_leaf, n_jobs=-1, class_weight = 'balanced_subsample')

    
    score = cross_val_score(regr, train_features, train_label, cv=5, scoring="balanced_accuracy")
    acc_mean = score.mean()

    return acc_mean

# Add stream handler of stdout to show the messages
# optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "random_forest-study" 
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage_name, load_if_exists=True)

[I 2022-09-16 10:28:41,452] Using an existing study with name 'random_forest-study' instead of creating a new one.


Using an existing study with name 'random_forest-study' instead of creating a new one.


In [ ]:
optuna.logging.enable_default_handler()
study.optimize(objective, n_trials=200)

[I 2022-09-16 10:47:26,785] Trial 2 finished with value: 0.9276979739261162 and parameters: {'criterion': 'log_loss', 'bootstrap': 'False', 'max_depth': 695, 'max_features': 'sqrt', 'min_samples_split': 9, 'min_samples_leaf': 6, 'n_estimators': 500}. Best is trial 2 with value: 0.9276979739261162.


Trial 2 finished with value: 0.9276979739261162 and parameters: {'criterion': 'log_loss', 'bootstrap': 'False', 'max_depth': 695, 'max_features': 'sqrt', 'min_samples_split': 9, 'min_samples_leaf': 6, 'n_estimators': 500}. Best is trial 2 with value: 0.9276979739261162.


In [ ]:
#Create an instance with tuned hyperparameters
optimised_rf = RandomForestClassifier(bootstrap = study.best_params['bootstrap'], criterion = study.best_params['criterion'],
                                     max_depth = study.best_params['max_depth'], max_features = study.best_params['max_features'],
                                     max_leaf_nodes = study.best_params['max_leaf_nodes'],n_estimators = study.best_params['n_estimators'],
                                     n_jobs=2)
#learn
optimised_rf.fit(X_train ,y_train)

In [ ]:
def objective(trial):
    
    regressor = trial.suggest_categorical('regressor', self._regressors)


    # TODO: make sure we do not hit boundaries!

    if regressor == 'RandomForest':


        # cf. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

        params = dict(

            oob_score=False, 

            max_depth=trial.suggest_int('max_depth', 1, 50),

            n_estimators=int(trial.suggest_discrete_uniform('n_estimators', 100, 1000, 100)),

            min_samples_split=trial.suggest_int('min_samples_split', 2, 10),

            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),

            max_features=trial.suggest_categorical('max_features', [None, "sqrt", "log2"])

        )


        model = RandomForestRegressor(**params, n_jobs=1)


    elif regressor == 'XGBoost':


        # cf. https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=n_estimators#module-xgboost.sklearn

        params = dict(

            booster=trial.suggest_categorical('booster', ["gbtree"]),

            learning_rate=trial.suggest_uniform('learning_rate', 0, 1),

            gamma=trial.suggest_uniform('gamma', 0, 10),

            max_depth=trial.suggest_int('max_depth', 1, 50),

            n_estimators=int(trial.suggest_discrete_uniform('n_estimators', 100, 1000, 100))

        )


        model = XGBRegressor(**params, n_jobs=1, importance_type='gain')


    elif regressor == 'LightGBM':


        # cf. https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

        params = dict(  

            learning_rate=trial.suggest_uniform('learning_rate', 0, 1),

            num_leaves=trial.suggest_int('num_leaves', 2, 50),

            max_depth=trial.suggest_int('max_depth', 1, 50),

            n_estimators=int(trial.suggest_discrete_uniform('n_estimators', 100, 1000, 100)),

            min_child_samples=trial.suggest_int('min_child_samples', 1, 10)

        )


        model = LGBMRegressor(**params, n_jobs=1, importance_type='gain')


    else:


        raise Exception('Invalid regressor. Only the following choices are valid: "RandomForest", "XGBoost", "LightGBM".')


    score = cross_val_score(model, trn_X, trn_y, n_jobs=max(self.n_cpus//2, 1), cv=cv)

    accuracy = score.min()


    return accuracy



study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=n_trials, n_jobs=max(self.n_cpus//2, 1))


# /!\ we need to make sure that type(n_estimators) == 'int'

best_params = study.best_params.copy()

best_params['n_estimators'] = int(best_params['n_estimators'])


print("Best parameters:", best_params)

print(f"Best score (minimum R2-score after {cv}-fold Cross Validation):", study.best_value)


# best_model = RandomForestRegressor(**best_params)

# best_model.fit(trn_X, trn_y)


# metrics = self._evaluate(best_model, trn_X, trn_y, tst_X, tst_y)

# print("Best model metrics:", json.dumps(metrics, indent=4))


self._best_params = study.best_params


return best_params